Basic Function for read and write graph

In [47]:
import sys
import numpy as np
from PNM import *
# define pi for convenience
pi = np.pi

def CreateAndSavePFM(out_path):
    width = 512
    height = 512
    numComponents = 3

    img_out = np.empty(shape=(width, height, numComponents), dtype=np.float32)
    
    for y in range(height):
        for x in range(width):
            img_out[y,x,:] = 1.0

    write_pfm(out_path, img_out)

def LoadAndSavePPM(in_path, out_path):
    img_in = read_ppm(in_path)
    img_out = np.empty(shape=img_in.shape, dtype=img_in.dtype)
    height,width,_ = img_in.shape # Retrieve height and width
    for y in range(height):
        for x in range(width):
            img_out[y,x,:] = img_in[y,x,:] # Copy pixels

    write_ppm(out_path, img_out)

def LoadAndSavePFM(in_path, out_path):
    img_in = read_pfm(in_path)
    img_out = np.empty(shape=img_in.shape, dtype=img_in.dtype)
    height,width,_ = img_in.shape # Retrieve height and width
    for y in range(height):
        for x in range(width):
            img_out[y,x,:] = img_in[y,x,:] # Copy pixels

    write_pfm(out_path, img_out)

def LoadPPMAndSavePFM(in_path, out_path):
    img_in = read_ppm(in_path)
    img_out = np.empty(shape=img_in.shape, dtype=np.float32)
    height,width,_ = img_in.shape
    for y in range(height):
        for x in range(width):
            img_out[y,x,:] = img_in[y,x,:]/255.0

    write_pfm(out_path, img_out)
            
def LoadPFMAndSavePPM(in_path, out_path):
    img_in = read_pfm(in_path)
    img_out = np.empty(shape=img_in.shape, dtype=np.float32)
    height,width,_ = img_in.shape
    for y in range(height):
        for x in range(width):
            img_out[y,x,:] = img_in[y,x,:] * 255.0

    write_ppm(out_path, img_out.astype(np.uint8))

def create_sphere_image():
    width = 511
    height = 511
    numComponents = 3

    img_out = np.empty(shape=(width, height, numComponents), dtype=np.float32)
    # the sphere has diameter 511, the center point is the center of the image, the other part outside the sphere is black
    for y in range(height):
        for x in range(width):
            if (x - 255.5) ** 2 + (y - 255.5) ** 2 <= 255.5 ** 2:
                img_out[y, x, :] = 1.0
            else:
                img_out[y, x, :] = 0.0

    return img_out


Load the environment graph and create the sphere

In [48]:
environment_graph = read_pfm('../Office/Office6.pfm')


sphere = create_sphere_image()
print(sphere.shape)
print(environment_graph.shape)

(511, 511, 3)
(699, 1242, 3)


Generate the vector map of the sphere

In [49]:
# store the normal vector of each pixel
normal = np.empty(shape=(sphere.shape[0], sphere.shape[1], 3), dtype=np.float32)
for y in range(sphere.shape[0]):
    for x in range(sphere.shape[1]):
        if sphere[y, x, 0] == 1.0:
            # move to the center of the sphere, y points up, x points right, z points out
            x_ = (x - 255.5) / 255.5
            y_ = -(y - 255.5) / 255.5
            z = np.sqrt(1 - x_ ** 2 - y_ ** 2)
            normal[y, x, :] = np.array([x_, y_, z])
# save as the graph
write_pfm( 'n_map.pfm', normal)

calculate the reflect vector with the normal vector and the view vector

In [50]:
v = np.array([0, 0, 1])
# r = 2*(n dot v)n – v
r_map = np.empty(shape=(sphere.shape[0], sphere.shape[1], 3), dtype=np.float32)
for y in range(sphere.shape[0]):
    for x in range(sphere.shape[1]):
        if sphere[y, x, 0] == 1.0:
            n = normal[y, x, :]
            r = 2 * np.dot(n, v) * n - v
            r_map[y, x, :] = r/np.linalg.norm(r)
write_pfm('../r_map.pfm', r_map)

In [51]:
# conver the normal map and r map to the ppm image
normal_ppm = np.empty(shape=(sphere.shape[0], sphere.shape[1], 3), dtype=np.uint8)
r_map_ppm = np.empty(shape=(sphere.shape[0], sphere.shape[1], 3), dtype=np.uint8)
for x in range(sphere.shape[0]):
    for y in range(sphere.shape[1]):
        if sphere[x, y, 0] == 1.0:
            # normal_ppm[x, y, :] = (normal[x, y, :] + 1 ) * 127.5 int 
            # r_map_ppm[x, y, :] = (r_map[x, y, :] + 1 ) * 127.5 int
            normal_ppm[x, y, :] = (normal[x, y, :]  + 1)/2 * 127.5
            r_map_ppm[x, y, :] = (r_map[x, y, :] + 1)/2 * 127.5
write_ppm('n_map.ppm', normal_ppm)
write_ppm('r_map.ppm', r_map_ppm)

We here define the x towards right, y points up, and z point out of the xy planer.
the phi $\phi$ is the angle between point in the projection of x-z planner and x axis. the theta $\theta$ is the angle between the point and the y axis.

In [52]:
def cartesian_to_spherical(x, y, z):
    theta = np.arccos(y)
    phi = np.arctan2(z, x)
    return theta, phi


In [53]:
sphere = create_sphere_image()
for y in range(sphere.shape[0]):
    for x in range(sphere.shape[1]):
        if sphere[y, x, 0] == 1.0:
            x_, y_, z = r_map[y, x, :]
            theta, phi = cartesian_to_spherical(x_, y_, z)
            theta = theta / np.pi 
            phi = (pi*3/2 - phi) / (2 * np.pi)
            if phi > 1:
                phi = phi - 1
            sphere[y, x, :] = environment_graph[min(environment_graph.shape[0]-1,int(theta * environment_graph.shape[0])), min(environment_graph.shape[1]-1,int(phi * environment_graph.shape[1])), :]
write_pfm('sphere.pfm', sphere)